## READ ME
Estos son los detalles de lo que hace el código:
1. importar librerías
2. importar data y realizar limpieza

    a. eliminar nulos (no afecta a las reservas)
    
    b. rellena los valores que tienen hora de reserva en blanco con el valor que se encuentra arriba
3. corregir typos básicos 
4. 

## READ ME
Estos son los detalles de lo que hace el código:
1. importar librerías
2. importar data y realizar limpieza

    a. eliminar nulos (no afecta a las reservas)
    
    b. rellena los valores que tienen hora de reserva en blanco con el valor que se encuentra arriba
3. verificar las direcciones correctas. Las que no estén verificadas se proceden a dejar fuera para que posteriormente se modifiquen manualmente 
4. Se calcula la distancia entre cada par de direcciones usando coordenadas (origen y destino)

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
import re


import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv(r'C:\Users\martin.olivares\Desktop\projects\best-route\test_3.csv')

df=pd.DataFrame()
df['address']=data['Direccion de inicio']
df['hora_recogida']=data['Hora de recogida']
df['destino']=data['Dirección destino']

df["num_empty_cells"] = df.isna().sum(axis=1)
df["nulls"]=df['num_empty_cells']/max(df['num_empty_cells'])

df = df.drop(df[df['nulls']==1].index)
df.fillna(method='ffill', inplace=True)

df.drop(columns=['nulls','num_empty_cells'],inplace=True)


In [3]:
# Definir un diccionario con las abreviaturas de calles y sus correspondientes formas completas
street_abbreviations = {
    "cl": "calle",
    "av": "avenida",
    "pj": "pasaje",
    "cam": "camino",
    "nte": "norte",
    "hermnos":'hnos',
    'hmnos':'hermanos',
    'tte':'teniente',
    }


# Definir una función que corrija las abreviaturas de calles en una dirección
def correct_typos(address):
    for abbreviation, full_form in street_abbreviations.items():
        address = re.sub(r'\b{}\b'.format(abbreviation), full_form, address)
    return address

# Aplicar la función a cada dirección del DataFrame
df["address"] = df["address"].str.lower().apply(correct_typos)
df['destino'] = df["destino"].str.lower().apply(correct_typos)

df


,address,hora_recogida,destino
0,"doctor israel roizblatt 130, valparaiso",6:20,"8 norte. 1168, viña del mar, valparaíso"
1,"ramaditas 1626, valparaíso",6:20,"8 norte. 1168, viña del mar, valparaíso"
2,"septimo de linea 17, valparaíso",6:20,"8 norte. 1168, viña del mar, valparaíso"
3,"dionisio hernández 450,viña del mar",6:30,"8 norte. 1168, viña del mar, valparaíso"
4,"camila 109, valparaíso",6:50,"8 norte. 1168, viña del mar, valparaíso"
5,"ramaditas 1626, valparaíso",20:40,"8 norte. 1168, viña del mar, valparaíso"
6,"doctor israel roizblatt 130, valparaiso",20:40,"8 norte. 1168, viña del mar, valparaíso"
7,"navío san martín 70, valparaíso",20:40,"8 norte. 1168, viña del mar, valparaíso"
8,"camila 109, valparaíso",20:40,"8 norte. 1168, viña del mar, valparaíso"
9,"viana 1155, valparaíso, viña del mar, valparaíso",20:40,"8 norte. 1168, viña del mar, valparaíso"


In [5]:
#Función para verificar que las direcciones estén correctas. Posteriormente se dejan fuera las incorrectas (momentaneo)

def correct_address(address):
    geolocator = Nominatim(user_agent="test")
    location = geolocator.geocode(address)
    if location:
        return location.address
    else:
        return np.nan


df['address'] = df['address'].apply(correct_address)
df['destino'] = df['destino'].apply(correct_address)

print('Numero de valores erroneos:')
print(df.isna().sum())


filter= df['address']!='no_existe_la_direccion'
df.where(filter ,inplace=True)

df.dropna(inplace=True)



df


Numero de valores erroneos:
address          1
hora_recogida    0
destino          3
dtype: int64


,address,hora_recogida,destino
0,"Doctor Israel Roizblatt, Población Dr. Israel ...",6:20,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
1,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",6:20,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
2,"Séptimo de Línea, Villa El Totoral, San Felipe...",6:20,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
3,"Dionisio Hernández, Población Nuevo Horizonte,...",6:30,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
4,"109, Camila, Cerro San Juan de Dios, Almendral...",6:50,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
5,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
6,"Doctor Israel Roizblatt, Población Dr. Israel ...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
7,"Navío San Martín, Bosque Inglés, San Roque, Va...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
8,"109, Camila, Cerro San Juan de Dios, Almendral...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d..."
9,"Viana, Población Saenz, Forestal, Viña del Mar...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d..."


In [ ]:
# Agrega la columna de etiquetas al dataframe
df['label'] = np.nan

# Agrupa las direcciones en función de su distancia usando KMeans
geolocator = Nominatim(user_agent="geoapiExercises")
grouped = df.groupby("hora_recogida")
for name, group in grouped:
    if len(group) > 1:
        # Calcula la distancia entre cada par de direcciones de recogida y destino
        X = np.zeros((len(group), len(group)))
        X_destino = np.zeros((len(group), len(group)))
        for i in range(len(group)):
            address1 = group.iloc[i]['address']
            loc1 = geolocator.geocode(address1)
            lat1, lon1 = loc1.latitude, loc1.longitude
            point1 = (lat1, lon1)
            for j in range(i+1, len(group)):
                address2 = group.iloc[j]['address']
                loc2 = geolocator.geocode(address2)
                lat2, lon2 = loc2.latitude, loc2.longitude
                point2 = (lat2, lon2)
                X[i, j] = great_circle(point1, point2).m
                X[j, i] = great_circle(point1, point2).m
                address3 = group.iloc[j]['destino']
                loc3 = geolocator.geocode(address3)
                lat3, lon3 = loc3.latitude, loc3.longitude
                point3 = (lat3, lon3)
                X_destino[i, j] = great_circle(point1, point3).m
                X_destino[j, i] = great_circle(point2, point3).m
        
        # Crea una matriz con las distancias de recogida y destino
        X_final = np.concatenate((X, X_destino), axis=1)
        X_final = np.concatenate((X_final, np.concatenate((np.transpose(X), np.transpose(X_destino)), axis=1)), axis=0)

        kmeans = KMeans(n_clusters=1)
        kmeans.fit(X_final)
        distances = cdist(X_final[kmeans.labels_ == 0], kmeans.cluster_centers_)
        cluster_labels = np.argmin(distances, axis=1)

        # Ajusta el número de clusters hasta que cada uno contenga 4 o menos elementos
        while np.unique(cluster_labels).size < kmeans.n_clusters:
            kmeans.n_clusters -= 1
            distances = cdist(X_final[kmeans.labels_ == 0], kmeans.cluster_centers_)
            cluster_labels = np.argmin(distances, axis=1)

        # Agrega las etiquetas al dataframe
        for i in range(len(group)):
            df.loc[group.index[i], 'label'] = cluster_labels[i]
# Agrupa las direcciones en función de su distancia usando KMeans
geolocator = Nominatim(user_agent="geoapiExercises")
grouped = df.groupby("hora_recogida")
for name, group in grouped:
    if len(group) > 1:
        # Calcula la distancia entre cada par de direcciones de recogida y destino
        X = np.zeros((len(group), len(group)))
        X_destino = np.zeros((len(group), len(group)))
        for i in range(len(group)):
            address1 = group.iloc[i]['address']
            loc1 = geolocator.geocode(address1)
            lat1, lon1 = loc1.latitude, loc1.longitude
            point1 = (lat1, lon1)
            for j in range(i+1, len(group)):
                address2 = group.iloc[j]['address']
                loc2 = geolocator.geocode(address2)
                lat2, lon2 = loc2.latitude, loc2.longitude
                point2 = (lat2, lon2)
                X[i, j] = great_circle(point1, point2).m
                X[j, i] = great_circle(point1, point2).m
                address3 = group.iloc[j]['destino']
                loc3 = geolocator.geocode(address3)
                lat3, lon3 = loc3.latitude, loc3.longitude
                point3 = (lat3, lon3)
                X_destino[i, j] = great_circle(point1, point3).m
                X_destino[j, i] = great_circle(point2, point3).m
        
        # Crea una matriz con las distancias de recogida y destino
        X_final = np.concatenate((X, X_destino), axis=1)
        X_final = np.concatenate((X_final, np.concatenate((np.transpose(X), np.transpose(X_destino)), axis=1)), axis=0)

        kmeans = KMeans(n_clusters=1)
        kmeans.fit(X_final)
        distances = cdist(X_final[kmeans.labels_ == 0], kmeans.cluster_centers_)
        cluster_labels = np.argmin(distances, axis=1)

        # Ajusta el número de clusters hasta que cada uno contenga 4 o menos elementos
        while np.unique(cluster_labels).size < kmeans.n_clusters:
            kmeans.n_clusters -= 1
            distances = cdist(X_final[kmeans.labels_ == 0], kmeans.cluster_centers_)
            cluster_labels = np.argmin(distances, axis=1)

        # Agrega las etiquetas al dataframe
        for i in range(len(group)):
            df.loc[group.index[i], 'label'] = cluster_labels[i]




In [ ]:
df[['hora_recogida', 'address', 'destino', 'label']]

,hora_recogida,address,destino,label
0,6:20,"Doctor Israel Roizblatt, Población Dr. Israel ...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
1,6:20,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
2,6:20,"Séptimo de Línea, Villa El Totoral, San Felipe...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
3,6:30,"Dionisio Hernández, Población Nuevo Horizonte,...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
4,6:50,"109, Camila, Cerro San Juan de Dios, Almendral...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
5,20:40,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
6,20:40,"Doctor Israel Roizblatt, Población Dr. Israel ...","8 Norte, Conjunto Residencial Benidorm, Viña d...",1.0
7,20:40,"Navío San Martín, Bosque Inglés, San Roque, Va...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
8,20:40,"109, Camila, Cerro San Juan de Dios, Almendral...","8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
9,20:40,"Viana, Población Saenz, Forestal, Viña del Mar...","8 Norte, Conjunto Residencial Benidorm, Viña d...",1.0


# ---------------------------------------------------------------------------------------------

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")

grouped = df.groupby("hora_recogida")

for name, group in grouped:

    # Calcula la distancia entre cada par de direcciones
    X = np.zeros((len(group), len(group)))
    for i in range(len(group)):
        address1 = group.iloc[i]['address']
        loc1 = geolocator.geocode(address1)
        lat1, lon1 = loc1.latitude, loc1.longitude
        point1 = (lat1, lon1)
        for j in range(i+1, len(group)):
            address2 = group.iloc[j]['address']
            loc2 = geolocator.geocode(address2)
            lat2, lon2 = loc2.latitude, loc2.longitude
            point2 = (lat2, lon2)
            X[i, j] = great_circle(point1, point2).m
            X[j, i] = great_circle(point1, point2).m

    # Crea una matriz con las distancias
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(X)
    group['label'] = kmeans.labels_

    # Continúa agrupando hasta que cada clúster tenga 4 o menos direcciones
    while group.groupby('label').agg({'address':'count'}).max().values[0] > 4:
        kmeans.n_clusters += 1
        kmeans.fit(X)
        group['label'] = kmeans.labels_
    df.loc[group.index, "label"] = group["label"]


In [ ]:
print("El numero de direcciones erroneas es: ",filter.tolist().count(False))

df

El numero de direcciones erroneas es:  0


,address,hora_recogida,destino,label
0,"Doctor Israel Roizblatt, Población Dr. Israel ...",6:20,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
1,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",6:20,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
2,"Séptimo de Línea, Villa El Totoral, San Felipe...",6:20,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
3,"Dionisio Hernández, Población Nuevo Horizonte,...",6:30,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
4,"109, Camila, Cerro San Juan de Dios, Almendral...",6:50,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
5,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
6,"Doctor Israel Roizblatt, Población Dr. Israel ...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d...",1.0
7,"Navío San Martín, Bosque Inglés, San Roque, Va...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
8,"109, Camila, Cerro San Juan de Dios, Almendral...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d...",0.0
9,"Viana, Población Saenz, Forestal, Viña del Mar...",20:40,"8 Norte, Conjunto Residencial Benidorm, Viña d...",1.0
